<a href="https://colab.research.google.com/github/Munna-Prasad-Gupta/nlp/blob/main/Sentiment_Analysis_Deforestation_Distilbert_Sieberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.metrics import accuracy_score, f1_score

# Load the dataset
data_path = 'Mock_Public_Sentiment_Dataset__1000_rows_.csv'  # Update this with your file path
df = pd.read_csv(data_path)

# Select relevant columns and drop missing values
df = df[['Post/Response Text', 'Sentiment']].dropna()

# Map sentiments to numerical labels
label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
df['Sentiment'] = df['Sentiment'].map(label_mapping)

# Split the data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Post/Response Text'], df['Sentiment'], test_size=0.2, random_state=42
)

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

train_data = Dataset.from_dict({'text': train_texts.tolist(), 'label': train_labels.tolist()})
test_data = Dataset.from_dict({'text': test_texts.tolist(), 'label': test_labels.tolist()})

train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

# Set format for PyTorch tensors
train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

# Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {'accuracy': acc, 'f1': f1}

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to="none"  # Disable W&B integration
)



# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-ad58b6473ed5>:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.095900,1.099191,0.335000,0.201067
2,1.089000,1.099426,0.370000,0.273333
3,1.085200,1.097448,0.370000,0.273333


Evaluation Results: {'eval_loss': 1.0994261503219604, 'eval_accuracy': 0.37, 'eval_f1': 0.2733333333333333, 'eval_runtime': 0.7691, 'eval_samples_per_second': 260.055, 'eval_steps_per_second': 16.904, 'epoch': 3.0}


In [38]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# Suppress WandB logging if not required
os.environ["WANDB_DISABLED"] = "true"

# Load Dataset
df = pd.read_csv("deforestation_sentiment_dataset.csv")
print(len(df))
# Split Dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['sentiment'].tolist(), test_size=0.2, random_state=42
)

# Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Custom Dataset Class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

train_dataset = SentimentDataset(train_texts, train_labels)
val_dataset = SentimentDataset(val_texts, val_labels)

# Load Pre-trained Model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",  # Updated parameter name
    save_strategy="epoch",
    run_name="distilbert_deforestation_sentiment",  # Custom run name
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy="epoch",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the Model
trainer.train()

# Evaluate the Model
predictions = trainer.predict(val_dataset)
pred_labels = predictions.predictions.argmax(-1)
f1 = f1_score(val_labels, pred_labels, average='weighted')
print("F1 Score:", f1)

print(classification_report(val_labels, pred_labels, target_names=['Negative', 'Neutral', 'Positive']))

# Save the Model
model.save_pretrained('./fine_tuned_distilbert')
tokenizer.save_pretrained('./fine_tuned_distilbert')


1098


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.258000,0.155315
2,0.114900,0.140538
3,0.113600,0.133767


F1 Score: 0.9453296672970442
              precision    recall  f1-score   support

    Negative       0.90      0.99      0.94        79
     Neutral       0.96      0.97      0.96        66
    Positive       1.00      0.88      0.94        75

    accuracy                           0.95       220
   macro avg       0.95      0.95      0.95       220
weighted avg       0.95      0.95      0.95       220



('./fine_tuned_distilbert/tokenizer_config.json',
 './fine_tuned_distilbert/special_tokens_map.json',
 './fine_tuned_distilbert/vocab.txt',
 './fine_tuned_distilbert/added_tokens.json')

In [40]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

# Suppress WandB logging
os.environ["WANDB_DISABLED"] = "true"

# Load Dataset
df = pd.read_csv("deforestation_sentiment_dataset.csv")

# Preprocess Dataset
df['sentiment'] = df['sentiment'].astype(int)

# Stratified Split for Training and Validation Sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(),
    df['sentiment'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['sentiment']
)

# Load Tokenizer for Siebert's Model
tokenizer = RobertaTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")

# Custom Dataset Class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx],
        }

# Create Datasets
train_dataset = SentimentDataset(train_texts, train_labels)
val_dataset = SentimentDataset(val_texts, val_labels)

# Load Pre-trained Siebert Model with Fix
model = RobertaForSequenceClassification.from_pretrained(
    "siebert/sentiment-roberta-large-english",
    num_labels=3,  # Match the number of sentiment classes
    ignore_mismatched_sizes=True  # Allow resizing of the classifier layer
)

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.1,
    load_best_model_at_end=True,
    save_total_limit=1,
    report_to="none",  # Disable WandB
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the Model
trainer.train()

# Evaluate the Model
predictions = trainer.predict(val_dataset)
pred_labels = predictions.predictions.argmax(-1)

# Calculate Metrics
f1 = f1_score(val_labels, pred_labels, average='weighted')
classification_report_data = classification_report(val_labels, pred_labels, target_names=['Negative', 'Neutral', 'Positive'])

# Print Metrics
print(f"Weighted F1 Score: {f1}")
print(classification_report_data)

# Save the Fine-Tuned Model
model_save_path = "fine_tuned_siebert_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model saved at: {model_save_path}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at siebert/sentiment-roberta-large-english and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([3, 1024]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-40-eba2af3e935c>:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.250700,0.276859
2,0.154300,0.152449
3,0.102100,0.141925
4,0.048800,0.137785


Weighted F1 Score: 0.9278740762783316
              precision    recall  f1-score   support

    Negative       0.96      0.89      0.92        74
     Neutral       0.86      0.97      0.91        73
    Positive       0.99      0.92      0.95        73

    accuracy                           0.93       220
   macro avg       0.93      0.93      0.93       220
weighted avg       0.93      0.93      0.93       220

Model saved at: fine_tuned_siebert_model
